**Step 1: Install & Setup (Run Once)**

Kaggle uses Python 3.10, so the path for the sed fix is slightly different from Colab.

In [ ]:
import os
import torch

# GPU Check
assert torch.cuda.is_available(), "Please enable GPU P100 in Kaggle Settings!"

print("🛠️ Installing Real-ESRGAN and dependencies...")
!rm -rf Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

# Install basicsr directly from the source to avoid some common bugs
!pip install -q facexlib gfpgan ffmpeg-python
!pip install -q git+https://github.com/XPixelGroup/BasicSR.git
!pip install -q -r requirements.txt
!python setup.py develop

# Modern Torchvision Patch for Python 3.12+
!find /usr/local/lib/python3.*/dist-packages/basicsr -name "*.py" -exec sed -i 's/torchvision.transforms.functional_tensor/torchvision.transforms.functional/g' {} +

print("✅ Environment is ready and patched!")

**Step 2: Full UI for Inputs**

Run this cell to see the dropdown menus. Select your settings, then move to Step 3.

In [ ]:
import cv2
import os
import ipywidgets as widgets
from IPython.display import display

# UI Elements
video_input = widgets.Text(value='/kaggle/input/your-video/fox-playing-mbl.mp4', description='Video Path:')
res_choice = widgets.Dropdown(
    options=["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)", "8k (7680 x 4320)", "2x Original", "3x Original", "4x Original"],
    value="2x Original",
    description='Resolution:'
)
model_choice = widgets.Dropdown(
    options=["RealESRGAN_x4plus", "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"],
    value="RealESRGAN_x4plus",
    description='Model:'
)

print("⚙️ CONFIGURE YOUR SETTINGS:")
display(video_input, res_choice, model_choice)

**Step 3: Resolution Logic & Processing**

This block reads the values you selected in the UI above and starts the P100-optimized upscale.

In [ ]:
import subprocess

# 1. Setup paths
video_path = video_input.value
resolution = res_choice.value
model_name = model_choice.value
output_dir = "/kaggle/working/"

if not os.path.exists(video_path):
    print(f"❌ ERROR: File not found at {video_path}")
else:
    # 2. Resolution Logic
    cap = cv2.VideoCapture(video_path)
    w, h = int(cap.get(3)), int(cap.get(4))
    cap.release()

    res_map = {
        "FHD (1920 x 1080)": (1920, 1080), "2k (2560 x 1440)": (2560, 1440),
        "4k (3840 x 2160)": (3840, 2160), "8k (7680 x 4320)": (7680, 4320),
        "2x Original": (w*2, h*2), "3x Original": (w*3, h*3), "4x Original": (w*4, h*4)
    }
    target_w, target_h = res_map[resolution]
    if w < h and "Original" not in resolution:
        if target_w > target_h: target_w, target_h = target_h, target_w
    scale_factor = max(target_w/w, target_h/h)

    # 3. Execution - P100 OPTIMIZED
    # We use -t 1024 because P100 has 16GB VRAM. This reduces "Tile" overhead.
    # We ensure --fp32 is NOT present so the engine uses half-precision (FP16).
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    raw_output = os.path.join(output_dir, f"{base_name}_out.mp4")
    final_output = os.path.join(output_dir, f"{base_name}_Upscaled.mp4")

    upscale_cmd = (
        f"python inference_realesrgan_video.py "
        f"-n {model_name} -i '{video_path}' -o '{output_dir}' "
        f"-s {scale_factor} -t 1024"
    )

    print(f"🚀 Upscaling to {target_w}x{target_h} using P100 Tensor Cores...")
    !{upscale_cmd}

    # 4. Final Fast Encode
    if os.path.exists(raw_output):
        print("🎬 Finalizing video...")
        ffmpeg_cmd = f"ffmpeg -loglevel error -y -i '{raw_output}' -c:v libx264 -preset ultrafast -crf 20 -pix_fmt yuv420p '{final_output}'"
        subprocess.run(ffmpeg_cmd, shell=True)
        print(f"✨ ALL DONE! Check /kaggle/working/ for: {os.path.basename(final_output)}")
        os.remove(raw_output)